<a href="https://colab.research.google.com/github/AtharKharal/Data_science_with_python/blob/dev-branch/featuretools_tpot_workflow_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Module Imports

In [0]:
#!pip install tornado==4.5.3 featuretools==0.6.1 pandas==0.24.1

#numpy==1.14.6

In [0]:
import numpy as np
import pandas as pd
import featuretools as ft
from featuretools.primitives import make_trans_primitive
import featuretools.variable_types as vt

#Data Reading

In [0]:
# Go to the CSV file in Google Drive, right-click on it and select 
# “Get shareable link”. The link will be copied into your clipboard. 
# Paste this link into following a string variable i.e. link:

link = 'https://drive.google.com/open?id=10JNCo2Opquuq2cwGiyGgvlds8U2wz3mW'  # The shareable link

# Code to read csv file into Colaboratory:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


fluff, id = link.split('=')   # Verify that you have everything after '=' by using print(id)

# Finally reading the data into a dataframe

downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('dummy_file_name.csv')  
df1 = pd.read_csv('dummy_file_name.csv')

#Featuretools

In [0]:
df1 = df1.iloc[:,1:df1.shape[1]]   # Target Variable must be removed for feature augmentation
df1['df_index']= np.arange(df1.shape[0])   # df_index is index for dataframe, if available; good, otherwise provide it as given here

#df1 = pd.DataFrame({'df_index' : [1,2,3,4,5],
#                 'location':['aust','aust','aust','canada','canada'],
#                  'prices':[34,52,46,25,67],
#                   'values':[786,345,123,654,841],
#                    'cat':[1,1,2,1,2]
#                  })


In [0]:
df1.head()

,Account Balance,Duration of Credit (month),Payment Status of Previous Credit,Purpose,Credit Amount,Value Savings/Stocks,Length of current employment,Instalment per cent,Sex & Marital Status,Guarantors,...,Most valuable available asset,Age (years),Concurrent Credits,Type of apartment,No of Credits at this Bank,Occupation,No of dependents,Telephone,Foreign Worker,df_index
0,1,18,4,2,1049,1,2,4,2,1,...,2,21,3,1,1,3,1,1,1,0
1,1,9,4,0,2799,1,3,2,3,1,...,1,36,3,1,2,3,2,1,1,1
2,2,12,2,9,841,2,4,2,2,1,...,1,23,3,1,1,2,1,1,1,2
3,1,12,4,0,2122,1,3,3,3,1,...,1,39,3,1,2,2,2,1,2,3
4,1,12,4,0,2171,1,3,4,3,1,...,2,38,1,2,2,2,1,1,2,4


In [0]:
def log10(column):
  return np.log10(column)

Log10 = make_trans_primitive(function=log10,
                             input_types=[vt.Numeric],
                             return_type=vt.Numeric)

In [0]:
#df1['date']=pd.date_range(start='02/07/2019',periods=5, freq='D')

es = ft.EntitySet(id='myEntitySet_id')
es.entity_from_dataframe(dataframe=df1,
                         entity_id='df1_id',
                         index='df_index'
                        )
# variable_types={'location': vt.Categorical,
#                 'prices':vt.Numeric,
#                 'values':vt.Numeric,
#                 'date':vt.Datetime
#           }, 

es.normalize_entity(base_entity_id='df1_id', new_entity_id='df2_id', 
                   index= 'Purpose' )

Entityset: myEntitySet_id
  Entities:
    df1_id [Rows: 1000, Columns: 21]
    df2_id [Rows: 10, Columns: 1]
  Relationships:
    df1_id.Purpose -> df2_id.Purpose

In [0]:
fm, features = ft.dfs(entityset=es, target_entity='df1_id',
                      verbose = True                        
                     )
#max_depth = 2, n_jobs = 1 ,trans_primitives = ['add', 'multiply', Log10], agg_primitives = ['sum', 'mean'],


Built 135 features
Elapsed: 00:03 | Remaining: 00:00 | Progress: 100%|██████████| Calculated: 10/10 chunks


In [0]:
features

#Tpot